<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n70_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n70_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...          y_hat         e_hat
rep  index                        ...                             
0.0  0.0    39.072153  40.249346  ...  141493.271404 -1.101802e+06
     1.0    19.043452  36.733115  ...  -46423.152112 -1.624381e+04
     2.0    38.887561  40.846929  ...  134037.752341 -1.140567e+06
     3.0    48.688138  40.941171  ...  233526.801925 -2.646034e+06
     4.0    46.098840  36.359158  ...  210044.940676  3.121817e+06
...               ...        ...  ...            ...           ...
99.0 65.0   37.864487  25.192202  ...  143366.738070  3.959259e+06
     66.0   21.268174  21.406557  ...  -17874.692692  3.302912e+05
     67.0   39.929908  22.174996  ...  166052.097965  2.201354e+05
     68.0   42.044898  22.342722  ...  188043.105072 -6.339259e+06
     69.0   44.311612  22.920802  ...  204129.016813  9.326472e+05

[7000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...          y_hat         e_hat
rep  index                        ...                             
0.0  0.0    39.072153  40.249346  ...  141493.271404 -1.101802e+06
     1.0    19.043452  36.733115  ...  -46423.152112 -1.624381e+04
     2.0    38.887561  40.846929  ...  134037.752341 -1.140567e+06
     3.0    48.688138  40.941171  ...  233526.801925 -2.646034e+06
     4.0    46.098840  36.359158  ...  210044.940676  3.121817e+06
...               ...        ...  ...            ...           ...
99.0 65.0   37.864487  25.192202  ...  143366.738070  3.959259e+06
     66.0   21.268174  21.406557  ...  -17874.692692  3.302912e+05
     67.0   39.929908  22.174996  ...  166052.097965  2.201354e+05
     68.0   42.044898  22.342722  ...  188043.105072 -6.339259e+06
     69.0   44.311612  22.920802  ...  204129.016813  9.326472e+05

[7000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,39.072153,40.249346,24.443951,-9.607694e+05,-9.603090e+05,141493.271404,-1.101802e+06
1,0.0,1.0,19.043452,36.733115,34.751846,-6.293388e+04,-6.266696e+04,-46423.152112,-1.624381e+04
2,0.0,2.0,38.887561,40.846929,14.345126,-1.006979e+06,-1.006530e+06,134037.752341,-1.140567e+06
3,0.0,3.0,48.688138,40.941171,21.349709,-2.413062e+06,-2.412507e+06,233526.801925,-2.646034e+06
4,0.0,4.0,46.098840,36.359158,17.164177,3.331343e+06,3.331862e+06,210044.940676,3.121817e+06
...,...,...,...,...,...,...,...,...,...
6995,99.0,65.0,37.864487,25.192202,25.487015,4.102191e+06,4.102626e+06,143366.738070,3.959259e+06
6996,99.0,66.0,21.268174,21.406557,21.627969,3.121558e+05,3.124165e+05,-17874.692692,3.302912e+05
6997,99.0,67.0,39.929908,22.174996,25.112466,3.857359e+05,3.861875e+05,166052.097965,2.201354e+05
6998,99.0,68.0,42.044898,22.342722,27.905088,-6.151691e+06,-6.151216e+06,188043.105072,-6.339259e+06


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...          y_hat         e_hat
rep  index                        ...                             
0.0  0.0    39.072153  40.249346  ...  141493.271404 -1.101802e+06
     1.0    19.043452  36.733115  ...  -46423.152112 -1.624381e+04
     2.0    38.887561  40.846929  ...  134037.752341 -1.140567e+06
     3.0    48.688138  40.941171  ...  233526.801925 -2.646034e+06
     4.0    46.098840  36.359158  ...  210044.940676  3.121817e+06
...               ...        ...  ...            ...           ...
99.0 65.0   37.864487  25.192202  ...  143366.738070  3.959259e+06
     66.0   21.268174  21.406557  ...  -17874.692692  3.302912e+05
     67.0   39.929908  22.174996  ...  166052.097965  2.201354e+05
     68.0   42.044898  22.342722  ...  188043.105072 -6.339259e+06
     69.0   44.311612  22.920802  ...  204129.016813  9.326472e+05

[7000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
1.0,23.0,41.0,831263086367.0421,241867814719852.97,37784685743.95646,6046695367996.324,163.22383017851743,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
4.0,29.0,35.0,43559809119288.3,210552193359827.38,1555707468546.0105,6192711569406.6875,4.005011389357764,0.00025664462652907893,0.9998716776867355,Reject005=0 : Heteroscedasticity
6.0,35.0,29.0,1453554516140.007,636820888873198.0,42751603415.88256,22743603174042.785,528.7568689807114,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
7.0,37.0,27.0,18975305878778.17,449247301112230.6,527091829966.0603,17278742350470.408,32.44401931362688,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
11.0,25.0,39.0,79642571653417.83,603541526337512.9,3318440485559.076,15882671745724.023,4.857773748061725,9.143016212576072e-05,0.9999542849189371,Reject005=0 : Heteroscedasticity
13.0,29.0,35.0,5788066048753.543,498123371670717.4,206716644598.34082,14650687402079.922,71.30720178269291,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
15.0,40.0,24.0,104936295509737.06,116978517673915.08,2690674243839.4116,5086022507561.525,1.8579290909416657,0.08109443921459336,0.9594527803927033,Reject005=1 : Homoscedasticity
18.0,29.0,35.0,18848553502338.87,262113675636811.6,673162625083.5311,7709225754023.87,11.522364442637558,1.911329094994585e-09,0.9999999990443355,Reject005=0 : Heteroscedasticity
20.0,31.0,33.0,118353198403065.14,232252054654614.88,3945106613435.505,7257876707956.715,1.843432839147377,0.09032463344692121,0.9548376832765394,Reject005=1 : Homoscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,42.0,22.0,610304086305374.0,1724387338258.6357,14885465519643.268,82113682774.22075,0.00539405235044392,1.4282636430791366e-20,Reject005=0 : Heteroscedasticity
1.0,23.0,41.0,831263086367.0421,241867814719852.97,37784685743.95646,6046695367996.324,163.22383017851743,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
2.0,30.0,34.0,184085848868467.03,132510022095907.31,6347787892016.1045,4015455215027.494,0.6351417474529778,0.20077308944722227,Reject005=1 : Homoscedasticity
3.0,37.0,27.0,198834765104165.38,43620823628186.12,5523187919560.149,1677723985699.466,0.30063497296311464,0.001740054478376181,Reject005=0 : Heteroscedasticity
4.0,29.0,35.0,43559809119288.3,210552193359827.38,1555707468546.0105,6192711569406.6875,4.005011389357764,0.00025664462652907893,Reject005=0 : Heteroscedasticity
5.0,31.0,33.0,186441167180069.12,8553517051166.285,6214705572668.971,267297407848.9464,0.043097359879794274,1.2242056470213468e-14,Reject005=0 : Heteroscedasticity
6.0,35.0,29.0,1453554516140.007,636820888873198.0,42751603415.88256,22743603174042.785,528.7568689807114,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
7.0,37.0,27.0,18975305878778.17,449247301112230.6,527091829966.0603,17278742350470.408,32.44401931362688,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
8.0,33.0,31.0,397173855558859.56,942340846351.5902,12411682986214.361,31411361545.053005,0.0025256874689691547,1.1770779837078886e-32,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    72
Reject005=1 : Homoscedasticity      28
Name: Result_test, dtype: int64